In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_ads_details = pd.read_csv('avisos.csv')

In [3]:
data_ads_details.drop(columns=['idpais'],inplace = True)

In [4]:
data_ads_details.head()

,idaviso,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial
1,17903700,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud
2,1000150677,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,Full-time,Senior / Semi-Senior,Transporte
3,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción


In [5]:
views = pd.read_csv('vistas.csv')
views.sample(5)

,idaviso,idpostulante,cantVistas
3316444,1112457316,W9PEDlB,1
1258436,1112380204,ekNNpv2,1
964647,1112360063,bOJ8KEE,1
27496,1110982686,pzNWREr,2
2363210,1112428511,owzbPG4,1


In [9]:
postulants = pd.read_csv('postulantes.csv')

/home/agustin/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
#el Nombre ya esta codificado con nombre_sort
postulants.drop(columns=['nombre'],inplace = True)

In [11]:
postulants.head()

,idpostulante,estado,nombre_sort,fechanacimiento,sexo
0,4rP6q15,En Curso,6.0,NaN,NO_DECLARA
1,akbv6zm,Graduado,6.0,1994-06-11,FEM
2,EVe2kz,En Curso,6.0,1974-06-28,FEM
3,xkPqMKa,Graduado,6.0,1969-02-03,MASC
4,bOj3Mjd,Graduado,6.0,1985-10-18,MASC


In [12]:
postulants.count()

idpostulante       408146
estado             366395
nombre_sort        366395
fechanacimiento    388502
sexo               408146
dtype: int64

In [13]:
data_ads_details.count()

idaviso            19634
titulo             19634
descripcion        19634
nombre_zona        19634
tipo_de_trabajo    19634
nivel_laboral      19634
nombre_area        19634
dtype: int64

In [14]:
#Ahora me genero un dataframe con la misma cantidad de (idpotulante-idaviso) pero esta vez de NO postulaciones
import random

def get_random_ids(df_postulants,df_ads):
    dic = {'idpostulante':[], 'idaviso':[]}
    for i in range (0,3000000):#6268564 numero que deberia ser, lo cambio a uno mas chico para poder trabajar
        numero = random.randrange(0,408146)
        idpostulante = df_postulants['idpostulante'][numero]
        dic['idpostulante'].append(idpostulante)
        numero2 = random.randrange(0,19634)
        idaviso = df_ads['idaviso'][numero2]
        dic['idaviso'].append(idaviso)
    dataframe = pd.DataFrame(data=dic)
    return dataframe
  

In [15]:
random_postulant_ad = get_random_ids(postulants,data_ads_details)

In [16]:
random_postulant_ad.head()

,idaviso,idpostulante
0,1112270995,BmDNXav
1,1112230493,96X6G1v
2,1112415895,LNKmaE5
3,1111606349,Ezejk5J
4,1112237240,13mazL


In [17]:
random_postulant_ad = random_postulant_ad.sample(3000000)

In [18]:
#Mergeo los data frames sobre la informacion de edad y sexo y educacion del postulante del postulante con 
#la informacion del aviso en que se postulo tambien.
random_postulant_ad = random_postulant_ad.merge(postulants,on = 'idpostulante')
random_postulant_ad = random_postulant_ad.merge(data_ads_details,on = 'idaviso')

In [19]:
random_postulant_ad['se_postulo'] = 0

In [20]:
merge = views.merge(random_postulant_ad, how = 'right', on = ['idpostulante','idaviso'])

In [23]:
#Como es una postulacion consideramos (en caso contrario) que nunca visito 
#anuncio, es decir rellenamos con 0
merge['cantVistas']=merge['cantVistas'].fillna(0)

In [24]:
merge.head()

,idaviso,idpostulante,cantVistas,estado,nombre_sort,fechanacimiento,sexo,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,se_postulo
0,1000641565,RzrpMqR,3.0,En Curso,3.0,1995-12-15,FEM,VENDEDOR FULL TIME,<p> <strong>Importante empresa ubicada en la z...,Capital Federal,Full-time,Senior / Semi-Senior,Ventas,0
1,1002377536,1Y2o2Q,1.0,En Curso,0.0,1987-09-18,MASC,Personal con experiencia en lavandería para H...,"<p><strong><span lang=""ES"">Adecco Hoteles y Ca...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Mantenimiento,0
2,1002461239,aM4lPZ,1.0,En Curso,3.0,1992-01-26,FEM,Analista de Administración de Personal,"<p style=""""><span style="""">Sanatorio ITOIZ se ...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Administración de Personal,0
3,1002490173,Y1r8Bq,1.0,Graduado,1.0,1990-12-01,MASC,Administrativa/o Comercial,<p>Somos una empresa que elabora y distribuye ...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Administración,0
4,1110478929,EzeY514,1.0,Abandonado,3.0,1981-02-24,MASC,Vendedor Viajante CÓRDOBA CAPITAL,<p>Somos una empresa multinacional que opera e...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,0


In [25]:
merge.to_csv("no_postulantes3M.csv",index = False)